# Classification : Multi-Class


## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

* 함수 만들기

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (2) 데이터로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/iris.csv"
data = pd.read_csv(path)
data.head()

## 2.데이터 준비

### (1) y 값을 0,1,2로 변환하기
(sparse_categorical_crossentropy 사용을 위해)

In [ ]:
data['Species'] = data['Species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
data.head()

### (2) 데이터 분할1 : x, y

In [ ]:
target = 'Species'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

## 3.방법① : y를 one-hot encoding

* y 가변수화
* 컴파일에서 loss = 'categorical_crossentropy'

### (1) y에 대한 가변수화
* tf.keras.utils.to_categorical
* pd.get_dummies

In [ ]:
y_c = to_categorical(y, 3)

In [ ]:
y_c[:5]

### (2) 데이터 분할2 : train : val

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y_c, test_size = .3, random_state = 2022)

### (3) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (4) 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model = Sequential()

# Layer 정의
layer = Dense(3, input_shape = (nfeatures,), activation = 'softmax')

# 모델에 추가
model.add(layer)

# 모델요약
model.summary()

### (5) compile + 학습

* lr과 learning_rate은 같은 의미 입니다. 그러나 향후 버전에서는 lr이 제외될 예정이라고 합니다.
* y를 one-hot encoding 했다면, loss fuction은 **categorical_crossentropy** 입니다.

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.1), loss='categorical_crossentropy')

history = model.fit(x_train, y_train, epochs = 100,
                    validation_split=0.2).history

* 학습결과 그래프

In [ ]:
dl_history_plot(history)

### (6) 예측 및 평가
* 예측 결과는 softmax로 변환된 값 입니다.

In [ ]:
pred = model.predict(x_val)
pred[:5]

* 행 별로 제일 큰 값을 찾아서 그에 맞게 숫자(0,1,2)로 변환 합시다.

In [ ]:
# 5개 행만 살펴보면
np.argmax(pred[:5], axis = 1)

In [ ]:
y_val

In [ ]:
# 전체에 적용해서 변환합시다.
pred_1 = pred.argmax(axis=1)
pred_1

* 실제값 y_val도 원래 대로 돌려 놓습니다.

In [ ]:
y_val[:5]

In [ ]:
y_val_1 = y_val.argmax(axis=1)
y_val_1

In [ ]:
print(confusion_matrix(y_val_1, pred_1))
print(classification_report(y_val_1, pred_1))

## 4.방법② : y 그대로 사용

* y는 그대로 사용. (주의! 아래 두가지를 꼭 확인해야 합니다.)
    * y가 숫자로 인코딩되어 있어야 합니다. 
    * 그리고 0부터 시작해야 합니다.
* 컴파일에서 loss = 'sparse_categorical_crossentropy'

### (1) 데이터 분할2 : train : val

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 2022)

### (2) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (3) 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model = Sequential()

# Layer 정의
layer = Dense(3, input_shape = (nfeatures,), activation = 'softmax')

# 모델에 추가
model.add(layer)

# 모델요약
model.summary()

### (4) compile + 학습

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.1), 
              loss='sparse_categorical_crossentropy')

history = model.fit(x_train, y_train, epochs = 50,
                    validation_split=0.2).history

* 학습결과 그래프

In [ ]:
dl_history_plot(history)

### (5) 예측 및 평가
* 예측 결과는 softmax로 변환된 값 입니다.

In [ ]:
pred = model.predict(x_val)
pred[:5]

* 행 별로 제일 큰 값을 찾아서 그에 맞게 숫자(0,1,2)로 변환 합시다.

In [ ]:
# 5개 행만 살펴보면
np.argmax(pred[:5], axis = 1)

In [ ]:
# 전체에 적용해서 변환합시다.
pred_1 = pred.argmax(axis=1)
pred_1

* 실제값 y_val은 0,1,2 로 된 1차원 값입니다. 

In [ ]:
y_val

In [ ]:
print(confusion_matrix(y_val, pred_1))
print(classification_report(y_val, pred_1))

## 5.실습 : 와인 품질 분류

위 두가지 방법 중 한가지만 사용하시면 됩니다.


### (1) 데이터 전처리

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/winequality-white.csv"
data = pd.read_csv(path)
data['quality'] = np.where(data['quality'] == 3, 4, np.where(data['quality'] == 9, 8, data['quality']))
data['quality'] = data['quality'] - 4
data.head()

* 범주가 5개 입니다.
    * 0 - 최하  ~  4 - 최상

In [ ]:
data['quality'].value_counts()

* y에 대한 전처리
    * 여기서는 y를 그대로 두고, (그래서, 0 ~ 4로 범주를 맞췄습니다.)
    * sparse_categorical_crossentropy 를 적용합시다.

* 데이터 분할

In [ ]:
target = 'quality'
x = data.drop(target, axis = 1)
y = data.loc[:, target]
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size= .3
                                                  , random_state = 2022)

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (2) 모델링

* 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model = Sequential([Dense(5, input_shape = (nfeatures,), activation = 'softmax')])

# 모델요약
model.summary()

* compile + 학습

In [ ]:
epochs = 50

model.compile(optimizer=Adam(learning_rate=0.01), 
              loss='sparse_categorical_crossentropy')

history = model.fit(x_train, y_train, epochs = epochs,
                    validation_split=0.2).history

* 학습결과 그래프

In [ ]:
dl_history_plot(history)

In [ ]:
pred = model.predict(x_val)
pred_1 = pred.argmax(axis=1)

In [ ]:
pred[:2]

In [ ]:
print(confusion_matrix(y_val, pred_1))
print(classification_report(y_val, pred_1))

### (3) 실습1

* 다음의 summary를 보고 모델을 설계하시오.

| Layer (type) | Output Shape | Param # | 옵션 |
| ---- | ---- | ---- | ----|
| dense (Dense) | (None, 16) | 192 | node, input_shape, activation = 'relu' |
| dense_1 (Dense) | (None, 5) | 85 | node, activation = 'softmax' |     

In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언



# 모델요약



### (4) 실습2

* 다음의 summary를 보고 모델을 설계하시오.

| Layer (type) | Output Shape | Param # | 옵션 |
| ---- | ---- | ---- | ---- |
| dense (Dense) | (None, 16) | 192 | node, input_shape, activation = 'relu' |
| dense_1 (Dense) | (None, 8) | 136 | node, activation = 'relu'  |
| dense_2 (Dense) | (None, 5) | 45 | node, activation = 'softmax'  |

In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언



# 모델요약



### (5) 실습3

* 이번에는 여러분이 원하는 대로 설계하고, 학습해 봅시다.


In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언



# 모델요약

